In [2]:
import numpy as np
import math
from sklearn.cross_validation import KFold
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
%matplotlib inline
import math
from sklearn.metrics import accuracy_score, average_precision_score, precision_score,recall_score,roc_curve,precision_recall_curve, auc,f1_score, confusion_matrix
import operator
from scipy.optimize import fmin_bfgs

In [3]:
from sklearn.datasets import fetch_mldata
import numpy as np
import os
from pylab import *
mnist = fetch_mldata('MNIST original')
mnist.data.shape
mnist.target.shape
np.unique(mnist.target)
X_m, y_m = mnist.data / 255., mnist.target
X_train, X_test = X_m[:50000], X_m[50000:]
y_train, y_test = y_m[:50000], y_m[50000:]
X_train.shape
y_train.shape
size=len(y_train)

In [4]:
X_train_m= []
Y_train_m=[]
X_test_m= []
Y_test_m=[]
for i in range(y_test.shape[0]):
    if y_test[i] == 0.0:
        X_train_m.append(X_test[i])
        Y_train_m.append(1);
    elif y_test[i] == 1.0:
        X_train_m.append(X_test[i])
        Y_train_m.append(2);
    elif y_test[i] == 2.0:
        X_train_m.append(X_test[i])
        Y_train_m.append(3);
X_train_m=np.array(X_train_m)
Y_train_m=np.array(Y_train_m)
print X_train_m.shape
print Y_train_m.shape
print set(Y_train_m)
for i in range(y_train.shape[0]):
    if y_train[i] == 0.0:
        X_test_m.append(X_test[i])
        Y_test_m.append(1);
    elif y_train[i] == 1.0:
        X_test_m.append(X_test[i])
        Y_test_m.append(2);
    elif y_train[i] == 2.0:
        X_test_m.append(X_test[i])
        Y_test_m.append(3);
X_test_m=np.array(X_test_m[5000:30000])
Y_test_m=np.array(Y_test_m[5000:30000])
print X_test_m.shape
print Y_test_m.shape
print set(Y_test_m)

(3147L, 784L)
(3147L,)
set([1, 2, 3])
(13623L, 784L)
(13623L,)
set([1, 2, 3])


In [4]:
def read_input(filename):
    input_data=[];
    f=open(filename);
    #f.readline();
    #f.readline();
    for line in f:
        input_data.append(line.rstrip('\n').split(','));
    return input_data;

def create_feature_matrix(input_data):
    x_list=[a[0:len(input_data[0])-1] for a in input_data];
    x=np.array([[1]+a[0:len(input_data[0])-1] for a in input_data]);
    #x=np.transpose(x)
    x= x.astype(np.float);
    return x

def create_y_matrix(input_data):
    y_list=[a[len(input_data[0])-1] for a in input_data];
    y_list_1=[];
    for i in y_list:
        if(i=='Iris-setosa'):
            y_list_1.append(1);
        elif(i=='Iris-versicolor'):
            y_list_1.append(2);
        elif(i=='Iris-virginica'):
            y_list_1.append(3);
        
            
    y=np.array(y_list_1);
    #y=np.transpose(y)
    y= y.astype(np.float);
    return y

In [11]:
#input_data=read_input('winequality-red.csv')
input_data=read_input('iris.dat')
X=create_feature_matrix(input_data)
Y=create_y_matrix(input_data)
print X.shape
print Y.shape

(150L, 5L)
(150L,)


In [5]:
def l_theta(x,y, theta,c=0 ):
    """m=y.shape[0];
    print theta.shape
    print x.shape
    print (np.dot(x,np.transpose(np.matrix(theta)))).shape
    h_t=h_theta(np.transpose(np.matrix(theta))*x);
    l_theta=np.sum((y*log(h_t))-((1-y)-log(1-h_t)));
    l_theta=1.*l_theta/m;
    return l_theta"""
    theta = theta.reshape((x.shape[1], y.shape[1]))
    y1=np.dot(x,theta);
    y1=y1-y1.min(axis=1)[:,np.newaxis]
    y1=np.exp(-y1);
    y1=y1/y1.sum(axis=1)[:,np.newaxis]
    y11=y1*y;
    log_theta=np.sum(np.log(y11.sum(axis=1)))
    y11=y11/y11.sum(axis=1)[:,np.newaxis]
    y1=y1-y11;
    log_theta=log_theta/-float(x.shape[0])
    return log_theta;

In [6]:
def gradient(x,y,theta, c=0.1):
    theta = theta.reshape((x.shape[1], y.shape[1]))
    y1=np.dot(x,theta);
    y1=y1-y1.min(axis=1)[:,np.newaxis]
    y1=np.exp(-y1);
    y1=y1/y1.sum(axis=1)[:,np.newaxis]
    y11=y1*y;
    y11=y11/y11.sum(axis=1)[:,np.newaxis]
    y1=y1-y11;
    grad=np.dot(np.transpose(x), y1);
    grad=grad/-float(x.shape[0])
    
    return grad.ravel();

In [7]:
def predict(x_test, y_test, theta, classarray):
    predicted_y1=[]
    theta = theta.reshape((x_test.shape[1], y_test.shape[1]))
    predicted_y=np.dot(x_test, theta)
    predicted_y=predicted_y-predicted_y.min(axis=1)[:,np.newaxis]
    predicted_y=np.exp(-predicted_y)
    predicted_y=predicted_y/predicted_y.sum(axis=1)[:,np.newaxis]
    #print predicted_y
    predicted_y2 = np.zeros((x_test.shape[0], len(classarray)), dtype=np.float64)
    predicted_y1=np.argmax(predicted_y, axis=1).squeeze()+1
    for i, a in enumerate(classarray):
        predicted_y2[predicted_y1==a, i]=1;
    return predicted_y2;    

In [8]:
def calc_theta(x,y, c =0):
    def optimize_theta(theta):
        return l_theta(x,y,theta,c)
    
    def optimize_grad(theta):
        return gradient(x,y,theta,c)
    
    initial_theta=np.zeros((x.shape[1]* y.shape[1],1));
    
    return fmin_bfgs(optimize_theta, initial_theta, fprime=optimize_grad)

In [9]:
def do_cross_validation(X, y,  classarray,n_folds, verbose=False):
    
    accuracies = []
    precisions =[]
    recalls=[]
    f1_scores=[]
    y_p=[]
    i=1;
    X1=X;
        
    Y1 = np.zeros((X1.shape[0], len(classarray)), dtype=np.float64)
    for i, a in enumerate(classarray):
        Y1[y==a, i]=1;
    print Y1
    cv = KFold(len(Y1), n_folds, shuffle=True, random_state=5)
    
    for train_idx, test_idx in cv:
        theta=calc_theta(X1[train_idx],Y1[train_idx]);
        #theta=theta[0].reshape((X1[train_idx].shape[1], Y1[train_idx].shape[1]))
        print theta
        predicted_y=predict(X1[test_idx], Y1[test_idx], theta, classarray);
        #print Y1[test_idx];
        #print predicted_y;
        acc = accuracy_score(Y1[test_idx], predicted_y)
        #c_matrix= confusion_matrix(Y1[test_idx], predicted_y)
        precision=precision_score(Y1[test_idx], predicted_y, average="weighted")
        recall=recall_score(Y1[test_idx], predicted_y, average="weighted")
        f_score=f1_score(Y1[test_idx], predicted_y, average="weighted")
        if(verbose == True):
            print('fold %d accuracy=%.4f, precision=%.4f, recall=%.4f, F-measure%.4f, confusion matrix=' % (i, acc, precision, recall, f_score));
            #print c_matrix
        i=i+1;
        accuracies.append(acc)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f_score)
        prredicted_y=[]
    avg=np.mean(accuracies);
    avg_precision=np.mean(precisions)
    avg_recall=np.mean(recalls)
    avg_f1_score=np.mean(f1_scores)
    return avg, avg_precision, avg_recall, avg_f1_score;

In [17]:
do_cross_validation(X, Y,[1,2,3],10, verbose=True)

Optimization terminated successfully.
         Current function value: 0.000001
         Iterations: 64
         Function evaluations: 66
         Gradient evaluations: 66
[ -165.3917522  -1425.22485774  1590.61660995  -307.84929072   192.84046732
   115.00882341  -804.50230232    54.2488825    750.2534198   1147.72165811
  -453.83500944  -693.88664873   528.16505481   723.36514621 -1251.53020102]
fold 2 accuracy=0.8667, precision=0.9048, recall=0.8667, F-measure0.8644, confusion matrix=
Optimization terminated successfully.
         Current function value: 0.041271
         Iterations: 63
         Function evaluations: 65
         Gradient evaluations: 65
[ -2.28620785 -17.39174918  19.67795703  -4.31613941   1.03062965
   3.28550977 -10.97051503   2.22319693   8.7473181   15.88590236
  -3.56008494 -12.32581742   7.35087587   4.3067968  -11.65767267]
fold 3 accuracy=1.0000, precision=1.0000, recall=1.0000, F-measure1.0000, confusion matrix=
Optimization terminated successfully.
      

(0.98000000000000009,
 0.98492063492063497,
 0.98000000000000009,
 0.97960461760461759)

In [22]:
X1=X;
Y1 = np.zeros((X1.shape[0], 3), dtype=np.float64)
for i, a in enumerate([1,2,3]):
    Y1[Y==a, i]=1;
theta=calc_theta(X1,Y1);
#theta=theta[0].reshape((X1[train_idx].shape[1], Y1[train_idx].shape[1]))
print theta
predicted_y=predict(X1, Y1, theta, [1,2,3]);
#print Y1[test_idx];
#print predicted_y;
print "Accuracy: %f" %accuracy_score(Y1, predicted_y)
print "Precision: %f" %precision_score(Y1, predicted_y, average="weighted")
print "Recall: %f" %recall_score(Y1, predicted_y, average="weighted")
print "F1 Score: %f" %f1_score(Y1, predicted_y, average="weighted")

Optimization terminated successfully.
         Current function value: 0.039662
         Iterations: 62
         Function evaluations: 64
         Gradient evaluations: 64
[ -2.44482501 -20.18460599  22.629431    -4.6648963    1.1010832
   3.5638131  -11.88180094   2.60127544   9.28052551  17.11865031
  -3.83475439 -13.28389592   7.88103212   5.22011308 -13.1011452 ]
Accuracy: 0.986667
Precision: 0.986667
Recall: 0.986667
F1 Score: 0.986667


In [ ]:
do_cross_validation(X_train_m, Y_train_m,[1,2,3],10, verbose=True)

[[ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 ..., 
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]]
Optimization terminated successfully.
         Current function value: 0.000014
         Iterations: 74
         Function evaluations: 76
         Gradient evaluations: 76
[ 0.  0.  0. ...,  0.  0.  0.]
fold 2 accuracy=0.9905, precision=0.9905, recall=0.9905, F-measure0.9905, confusion matrix=
Optimization terminated successfully.
         Current function value: 0.000023
         Iterations: 61
         Function evaluations: 62
         Gradient evaluations: 62
[ 0.  0.  0. ...,  0.  0.  0.]
fold 3 accuracy=0.9905, precision=0.9906, recall=0.9905, F-measure0.9905, confusion matrix=
Optimization terminated successfully.
Optimization terminated successfully.
         Current function value: 0.000011
         Iterations: 72
         Function evaluations: 73
         Gradient evaluations: 73
[ 0.  0.  0. ...,  0.  0.  0.]
fold 5 accuracy=0.9810, precision=0.9810, recall=0.9810, F-measure0.980

(0.98601455868971788,
 0.98613362687732153,
 0.98601455868971788,
 0.98600496677486082)

In [10]:
X1=X_train_m;
Y1 = np.zeros((X1.shape[0], 3), dtype=np.float64)
for i, a in enumerate([1,2,3]):
    Y1[Y_train_m==a, i]=1;
theta=calc_theta(X1,Y1);
#theta=theta[0].reshape((X1[train_idx].shape[1], Y1[train_idx].shape[1]))
print theta
predicted_y=predict(X1, Y1, theta, [1,2,3]);
#print Y1[test_idx];
#print predicted_y;
print "Accuracy: %f" %accuracy_score(Y1, predicted_y)
print "Precision: %f" %precision_score(Y1, predicted_y, average="weighted")
print "Recall: %f" %recall_score(Y1, predicted_y, average="weighted")
print "F1 Score: %f" %f1_score(Y1, predicted_y, average="weighted")

Optimization terminated successfully.
         Current function value: 0.000008
         Iterations: 80
         Function evaluations: 81
         Gradient evaluations: 81
[ 0.  0.  0. ...,  0.  0.  0.]
Accuracy: 1.000000
Precision: 1.000000
Recall: 1.000000
F1 Score: 1.000000
